In [ ]:
# THIS PROGRAM CALCULATES THE VELOCITY AND VISCOSITY PROFILES FOR LAMINAR FLOWING YIELD-POWER-LAW FLUIDS IN ECCENTRIC ANNULI• IT ALSO NUMERICALLY INTEGRATES THE VELOCITY PROFILE
# OVER AREA TO OBTAIN FLOW RATE
# BY MARK HACI 8/24/1988, LSU

In [ ]:
# In summary, the procedure for calculating the velocity distribution is as follows:
# 1. Use Eq. 3.20 and 3.21 to calculate eps_1 and eps_0
# 2. Determine delta epsilon and delta n based on the size of the grid network
# 3. Set coefficients P and Q to zero at the boundaries (eps=eps_i and eps_0)
# 4. Guess a velocity field
# 5. Calculate a viscosity field (Eq. 4.14 and so on) based on the latest velocity field
# 6. Compute coefficients Aj, Bj, Cj and Dj using Eqs. 4.18-4.21
# 7. Use recurrence equations 4.25 and 4.26 to compute coefficients P and Q from J=2 to M-1 for each column
# 8. Calculate a new velocity field employing Eq. 4.22 from J=M-1 to 2 (back substitution)
# 9. Check for convergence comparing old and new velocity fields
# 10. Repeat from step 5 until the velocity field converges.

In [1]:
import numpy as np
import pandas as pd
from math import log, sqrt, sin, cos, sinh, cosh, pi

In [123]:
# inputs

ngridx = 52
ngridy = 82

def arr2D(x=52, y=82):
    # return np.random.normal(loc=0, scale=1, size=(x, y))
    return np.ones((x, y))

def arr1D(y=82):
    # return np.random.normal(loc=0, scale=1, size=y)
    return np.ones(y)

a = arr2D()
b = arr2D()
c = arr2D()
d = arr2D()
garea = arr2D()

p = arr2D()
q = arr2D()
v = arr2D()
vn = arr2D()

v1 = arr2D()
vis1 = arr2D()
vis2 = arr2D()
gare = arr2D()

visp1 = arr2D()
visb1 = arr2D()
visp2 = arr2D()
visb2 = arr2D()

delx = arr1D()
totl = arr1D()

d2 = 5          # outer diameter, inch
d1 = 3.5        # inner diameter, inch
e = 0.0001      # dimensionless
xn = 1          # dimensionless
xk = 0.001      # consistency index, eq. cp
tauo = 12       # yield point, lb/100
rho = 9.5       # dummy variable, ppg

dpdl = 0.0001   # firc. press. loss gradient, psi/ft
qflow = 200      # expected flow rate, gpm

niter = 100
conlim1 = 0.001
conlim2 = 0.001

In [125]:
# INITIALIZING AND READING DATA

vislim = 1e-6
vguess = qflow / (2.448*(d2**2-d1**2))

if e < 0.0001:
    e = 0.0001
if e > 0.99:
    e = 0.99

zn = 1 / (2 - xn)

taud = 4.788 * tauo * ((rho/8.34)**xn * (d2*1.27)**(2*xn) / (xk/100)**2)**zn
fconst = 2262.06 * ((rho/8.34)**xn * (d2*1.27)**(2+xn) / (xk/100)**2)**zn
f = dpdl * fconst

vco = (xk/100 / (rho/8.34) / (d2*1.27)**xn)**zn
qconst = vco * (d2*1.27)**2 / 63.09
vconst = vco/30.48
vguess = vguess/vconst
viscon = (xk/100 * (rho/8.34)**(1-xn) * (d2*1.27)**(2-2*xn))**zn * 100

# CALCULATING THE PIPE RADIUS RATIO AND PRINT DATA

s = d1/d2

# CALCULATING THE BOUNDARIES OF THE PIPES IN BIPJLAR COORDINATES AND THE DIMENSIONS OF THE GRIDS

j1 = ngridy
j2 = j1 - 1
i1 = ngridx
i2 = i1 - 1

total = 0
totl[0] = 0

# DO 24
for j in range(ngridx):
    i = ngridx - j + 1
    x8 = (i+1) / i
    x9 = ngridx + 1
    delx[j+1] = log(x8)**e / ((log(x9))**e * ngridx**(1-e)) * pi
    total += delx[j+1]

# DO 23
for j in range(ngridx):
    delx[j+1] = delx[j+1] * pi / total
    totl[j+1] = totl[j] + delx[j+1]

delx[0] = delx[1]
delx[i1] = delx[i2]

xm = e * (d2-d1)/d2
x1 = (1- s*s - xm*xm)/2/s/xm
x2 = (1- s*s - xm*xm)/2/xm
eps1 = log(x1 + sqrt(x1**2-1))
eps2 = log(x2 + sqrt(x2**2-1))
dely = (eps1-eps2) / ngridy
dsi = sinh(eps2)

delx.shape, totl.shape

# STARTING LINE-BY-LINE METHOD, SETTING THE COEFFIIENTS AT THE BOUNDARIES

# DO 51
for i in range(1, i2):
    a[i, 0] = 1
    b[i, 0] = 0
    c[i, 0] = 0
    d[i, 0] = 0
    p[i, 0] = b[i, 0] / a[i, 0]
    q[i, 0] = d[i, 0] / a[i, 0]
    p[i, -1] = 1   # unreadable
    q[i, -1] = 1   # unreadable

# GUESSING A VELOCITY FIELD

# DO 52
for i in range(i1):
    for j in range(j1):
        v[i, j] = vguess
        if j==0 or j==j1-1:
            v[i, j] = 0

# INITILAZING FLOW RATE, PRINTING COUNTER, AND ITERATION COUNTER

qt1 = 0

# REPEATING TO OBTAIN AN MIMPROVES VELOCITY FIELD
# 100 CONTINUE

for iter in range(niter):

    # COMPUTING AN AVERAGE VELOCITY AT THE SOUTH-WEST CORNER OF EACH CONTROL-VOLUME
    # DO 53
    for i in range(i1):
        for j in range(j1):
            numer = (delx[i-1]*(v[i,j] + v[i,j-1]) + delx[i]*(v[i-1,j] + v[i-1,j-1]))
            denom = (delx[i] + delx[i-1])
            v1[i,j] = 0.5 *  numer / denom

    # CALCULATING VISCOSITY AT THE WEST FACE OF EACH CONTROL-VOLUME
    # DO 31
    for i in range(1, i1):
        for j in range(1, j1):

            dvdx1 = abs(v[i, j] - v[i-1, j] / (delx[i-1] + delx[i]) / 2)

            if j==j2:  # j==1 or
                dvdy1 = 4/3 * dvdy1
            else:
                dvdy1 = abs(v[i, j] - v[i, j-1]) / dely

            c1 = dvdx1**2 + dvdy1**2

            if c1 < vislim:
                c1 = vislim

            eta = totl[i-1]
            eps = eps2 + (j-2)*dely + dely/2
            xi = cosh(eps) - cos(eta)
            sh1 = xi * sqrt(c1)
            visb1[i, j] = taud * dsi / abs(sh1)

            if xn <= 1:
                visp1[i, j] = abs((dsi/sh1)**(1-xn))
            else:
                visp1[i, j] = abs((sh1/dsi)**(xn-1))

            vis1[i, j] = visp1[i, j] + visb1[i, j]

    # CALCULATING VISCOSITY AT THE SOUTH FACE OF EACH CONTROL-VOLUME
    # DO 32
    for i in range(1, i2):
        for j in range(1, j1):

            dvdx2 = abs(v1[i, j] - v1[i+1, j]) / delx[i]

            if j==j1: # or j==1:
                dvdy2 = dvdy2 * 2
            else:
                dvdy2 = abs(v[i, j] - v[i, j-1]) / dely

            c2 = dvdx2**2 + dvdy2**2

            if c2 < vislim:
                c2 = vislim

            eta = totl[i-1] + delx[i]/2
            eps = eps2 + (j-2)*dely

            if j==2:
                eps = eps + dely/4
            if j==j1:
                eps = eps - dely/4

            xi = cosh(eps) - cos(eta)
            sh2 = xi * sqrt(c2)

            visb2[i, j] = taud * dsi / abs(sh2)

            if xn <= 1:
                visp2[i, j] = abs((dsi/sh2)**(1-xn))
            else:
                visp2[i, j] = abs((sh2/dsi)**(xn-1))

            vis2[i, j] = visp2[i, j] + visb2[i, j]

    # COMPUTING THE COEFFICIENTS OF THE TOMA EQN.
    # DO 54
    for i in range(1, i2):
        for j in range(1, j2):

            dels = dely
            deln = dely
            dele = (delx[i] + delx[i+1]) / 2
            delw = (delx[i] + delx[i-1]) / 2

            if j==2:
                dels = dely/2
            if j==j2:
                deln = dely/2

            di = (dele + delw) / 2
            dj = (deln + dels) / 2

            an = vis2[i, j+1] * di / deln
            _as = vis2[i, j] * di / dels
            ae = vis1[i+1, j] * dj / dele

            aw = vis1[i+1,j] * dj / dele
            a[i, j] = an + _as + ae + aw
            b[i, j] = an
            c[i, j] = _as

            if i==1:
                v[i-1, j] = v[i, j]
            if i==i2:
                v[i+1, j] = v[i, j]

            eta = totl[i-1] + delx[i]/2
            eps = eps2 + (j-2)*dely + dely/2

            xi = cosh(eps) - cos(eta)
            gare[i, j] = dsi * dsi / xi / xi * di * dj

            d[i,j] = ae * v[i+1, j] + aw * v[i-1, j] + gare[i, j] * f
            p[i,j] = b[i,j] / (a[i,j] - c[i,j] * p[i,j-1])
            q[i,j] = (c[i,j] * q[i,j-1] + d[i,j]) / (a[i,j] - c[i,j] * p[i,j-1])

    # CALCULATING "NEW" VELOCITY FIELD
    # DO 55
    for i in range(1, i2):
        for n in range(1, j2):
            j = j2 - n # + 2. TODO why +2?
            vn[i, j] = p[i, j] * v[i, j+1] + q[i, j]

    # DO 555
    for i in range(1, i2):
        for n in range(1, j2):
            v[i,j] = vn[i,j]

    # COUNTING NUMBER OF ITERATIONS
    # iprint += 1
    iter += 1

    # CALCULATING FLOW RATE
    qt = 0
    qtvis = 0
    vist = 0
    area = 0

    # DO 57
    for i in range(1, i2):
        for j in range(1, j2):

            area += gare[i, j]
            florate = v[i, j] * gare[i, j]
            qt += florate

    qt = 2 * qt
    vavg = qt / 2 / area
    print(f'iter={iter}\t vislim={vislim}\t qt={qt}')

    # 70 block
    if abs((qt1-qt)/qt < conlim1):

        ##### 444 #######################
        vislim *= 1e-2
        converged = (vislim < 1e-50) or (abs((qtvis - qt) / qt) < conlim2)
        if converged:
            break
        else:
            qtvis = qt
        #################################
    else:
        qt1 = qt

print(f'converged! vislim={vislim}')

iter=1	 vislim=1e-06	 qt=214916084.99433985
iter=2	 vislim=1e-08	 qt=214916084.99433985
iter=3	 vislim=1e-10	 qt=214916084.99433985


/tmp/ipython-input-4055787928.py:208: RuntimeWarning: divide by zero encountered in scalar divide
  p[i,j] = b[i,j] / (a[i,j] - c[i,j] * p[i,j-1])
/tmp/ipython-input-4055787928.py:209: RuntimeWarning: divide by zero encountered in scalar divide
  q[i,j] = (c[i,j] * q[i,j-1] + d[i,j]) / (a[i,j] - c[i,j] * p[i,j-1])
/tmp/ipython-input-4055787928.py:216: RuntimeWarning: invalid value encountered in scalar multiply
  vn[i, j] = p[i, j] * v[i, j+1] + q[i, j]


iter=4	 vislim=1e-12	 qt=214916084.99433985
iter=5	 vislim=1e-14	 qt=214916084.99433985
iter=6	 vislim=1e-16	 qt=214916084.99433985
iter=7	 vislim=1e-18	 qt=214916084.99433985
iter=8	 vislim=1.0000000000000001e-20	 qt=214916084.99433985
iter=9	 vislim=1.0000000000000002e-22	 qt=214916084.99433985
iter=10	 vislim=1.0000000000000001e-24	 qt=214916084.99433985
iter=11	 vislim=1.0000000000000002e-26	 qt=214916084.99433985
iter=12	 vislim=1.0000000000000002e-28	 qt=214916084.99433985
iter=13	 vislim=1.0000000000000003e-30	 qt=214916084.99433985
iter=14	 vislim=1.0000000000000003e-32	 qt=214916084.99433985
iter=15	 vislim=1.0000000000000004e-34	 qt=214916084.99433985
iter=16	 vislim=1.0000000000000004e-36	 qt=214916084.99433985
iter=17	 vislim=1.0000000000000005e-38	 qt=214916084.99433985
iter=18	 vislim=1.0000000000000005e-40	 qt=214916084.99433985
iter=19	 vislim=1.0000000000000005e-42	 qt=214916084.99433985
iter=20	 vislim=1.0000000000000006e-44	 qt=214916084.99433985
iter=21	 vislim=1.00

In [130]:
# SUBROUTINE POWER

def power(xmax, ipower=0, powr=1):

    if xmax > 0.09999:
        if xmax < 1:
            pass
        else:
            xmax = 0.1 * xmax
            ipower = ipower + 1
            powr = powr / 0.1
    else:
        xmax = 10*xmax
        ipower = ipower - 1
        powr = powr * 0.1

    return xmax, ipower, powr


In [131]:
# PRINTING VELOCITY FIELD

for j in range(1, j1):
    for i in range(1, i1):
        v[i,j] = vconst * v[i,j]

jm = j1/2
vmax = v[2,int(jm)]
vmax, _, _ = power(vmax)

for n in range(1, j1):
    j = j1 - n + 1

# PRINTING VISCOSITY FIELD
# ... we can print those easily while calculation above